In [1]:
pip install torch razdel pymorphy2 nltk tqdm tensorboard

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
import re
import string
from collections import Counter
import nltk
from nltk.corpus import stopwords
from razdel import tokenize
from pymorphy2 import MorphAnalyzer
from sklearn.decomposition import PCA
from tqdm import tqdm
import csv
import os
from datetime import datetime

In [3]:
nltk.download('stopwords')

class TextPreprocessor:
    def __init__(self):
        self.morph = MorphAnalyzer()
        self.russian_stopwords = set(stopwords.words('russian'))
        self.custom_stopwords = {
            'это', 'то', 'как', 'так', 'и', 'в', 'над', 'к', 'до', 'не', 'на', 'но', 'за', 'то', 
            'же', 'вы', 'бы', 'по', 'только', 'его', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 
            'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'ли', 'если', 'уже', 'или', 
            'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 
            'там', 'потом', 'себя', 'ничего', 'который', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 
            'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 
            'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 
            'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 
            'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 
            'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 
            'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 
            'эту', 'моя', 'впрочем', 'хорошо', 'свою', 'этой', 'перед', 'иногда', 'лучше', 'чуть', 
            'том', 'нельзя', 'такой', 'им', 'более', 'всегда', 'конечно', 'вдруг', 'сегодня', 'тот'
        }
        self.all_stopwords = self.russian_stopwords.union(self.custom_stopwords)
    
    def clean_text(self, text):
        if not isinstance(text, str):
            return ""
        
        # Приведение к нижнему регистру
        text = text.lower()
        
        # Удаление email адресов
        text = re.sub(r'\S*@\S*\s?', '', text)
        
        # Удаление URL
        text = re.sub(r'http\S+', '', text)
        
        # Удаление цифр
        text = re.sub(r'\d+', '', text)
        
        # Удаление пунктуации и специальных символов
        text = text.translate(str.maketrans('', '', string.punctuation))
        
        # Удаление лишних пробелов
        text = re.sub(r'\s+', ' ', text).strip()
        
        return text
    
    def lemmatize_tokens(self, tokens):
        """Лемматизация токенов"""
        lemmas = []
        for token in tokens:
            # Пропускаем стоп-слова и короткие токены
            if (len(token) <= 2 or 
                token in self.all_stopwords or 
                not re.match(r'^[а-яё]+$', token)):
                continue
            
            try:
                parsed = self.morph.parse(token)[0]
                lemma = parsed.normal_form
                # Проверяем, что лемма состоит только из русских букв
                if re.match(r'^[а-яё]+$', lemma) and len(lemma) > 1:
                    lemmas.append(lemma)
            except:
                continue
        
        return lemmas
    
    def preprocess(self, text):
        cleaned_text = self.clean_text(text)
        
        tokens = [token.text for token in tokenize(cleaned_text)]
        
        lemmas = self.lemmatize_tokens(tokens)
        
        return lemmas

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Zver\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
class Vocabulary:
    """Класс для работы со словарем"""
    
    def __init__(self, min_freq=5):
        self.min_freq = min_freq
        self.word2idx = {}
        self.idx2word = {}
        self.word_freq = {}
        self.unknown_token = '<UNK>'
        self.padding_token = '<PAD>'
        
    def build_vocabulary(self, texts):
        """Построение словаря на основе текстов"""
        print("Построение словаря")
        
        # Подсчет частот слов
        word_counts = Counter()
        for text in tqdm(texts, desc="Подсчет частот слов"):
            word_counts.update(text)
        
        # Фильтрация по минимальной частоте
        filtered_words = {word: count for word, count in word_counts.items() 
                         if count >= self.min_freq}
        
        # Создание словаря
        self.word2idx = {self.padding_token: 0, self.unknown_token: 1}
        self.idx2word = {0: self.padding_token, 1: self.unknown_token}
        
        for idx, word in enumerate(filtered_words.keys(), start=2):
            self.word2idx[word] = idx
            self.idx2word[idx] = word
        
        self.word_freq = filtered_words
        
        print(f"Размер словаря: {len(self.word2idx)} слов")
        print(f"Наиболее частые слова: {list(filtered_words.items())[:10]}")
    
    def encode_text(self, text):
        """Кодирование текста в индексы"""
        return [self.word2idx.get(word, self.word2idx[self.unknown_token]) 
                for word in text if word in self.word2idx]
    
    def get_word_list(self):
        """Получение списка слов в порядке индексов"""
        words = []
        for i in range(len(self.idx2word)):
            words.append(self.idx2word[i])
        return words
    
    def __len__(self):
        return len(self.word2idx)

In [5]:
class CBOWModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear = nn.Linear(embedding_dim, vocab_size)
        
    def forward(self, inputs):
        embeds = self.embeddings(inputs) 
        embeds = torch.mean(embeds, dim=1)
        out = self.linear(embeds) # Предсказываем целевое слово
        return out

class SkipGramModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear = nn.Linear(embedding_dim, vocab_size)
        
    def forward(self, target):
        embeds = self.embeddings(target)
        out = self.linear(embeds) # Предсказываем контекстые слова
        return out

In [6]:
class Word2VecDataset(Dataset):
    """Датасет для Word2Vec моделей"""
    
    def __init__(self, texts, vocab, window_size=5, model_type='cbow'):
        self.data = []
        self.model_type = model_type
        self.window_size = window_size
        
        print(f"Создание датасета для {model_type.upper()}...")
        
        for text in tqdm(texts, desc="Обработка текстов"):
            encoded_text = vocab.encode_text(text)
            
            if len(encoded_text) < window_size * 2 + 1:
                continue
            
            if model_type == 'cbow':
                self._create_cbow_samples(encoded_text)
            else:
                self._create_skipgram_samples(encoded_text)
    
    def _create_cbow_samples(self, encoded_text):
        """Создание samples для CBOW"""
        for i in range(self.window_size, len(encoded_text) - self.window_size):
            context = (encoded_text[i-self.window_size:i] + 
                      encoded_text[i+1:i+self.window_size+1])
            target = encoded_text[i]
            self.data.append((torch.tensor(context), torch.tensor(target)))
    
    def _create_skipgram_samples(self, encoded_text):
        """Создание samples для SkipGram"""
        for i in range(self.window_size, len(encoded_text) - self.window_size):
            target = encoded_text[i]    # Целевое слово
            # Для каждого слова в окне создаем пару (target, context_word)
            for j in range(i-self.window_size, i+self.window_size+1):
                if j != i and 0 <= j < len(encoded_text):
                    self.data.append((torch.tensor(target), torch.tensor(encoded_text[j])))
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]

In [7]:
class Word2VecTrainer:
    """Класс для обучения моделей Word2Vec"""
    
    def __init__(self, vocab_size, embedding_dim=100, learning_rate=0.001, weight_decay=1e-5, log_dir=None):
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.learning_rate = learning_rate
        self.writer = SummaryWriter(log_dir) if log_dir else None
        
    def train_cbow(self, dataloader, epochs=10, model_name="CBOW"):
        """Обучение CBOW модели"""
        print(f"Обучение {model_name} модели")
        model = CBOWModel(self.vocab_size, self.embedding_dim)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=self.learning_rate)
        
        model.train()
        for epoch in range(epochs):
            total_loss = 0
            progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}")
            
            for batch_idx, (context, target) in enumerate(progress_bar):
                optimizer.zero_grad()
                output = model(context)
                loss = criterion(output, target)
                loss.backward()
                optimizer.step()
                
                total_loss += loss.item()
                progress_bar.set_postfix({'loss': f'{loss.item():.4f}'})
                
                if self.writer and batch_idx % 100 == 0:
                    self.writer.add_scalar(f'{model_name}/batch_loss', loss.item(), 
                                         epoch * len(dataloader) + batch_idx)
            
            avg_loss = total_loss / len(dataloader)
            print(f'Epoch {epoch+1}, Average Loss: {avg_loss:.4f}')

            if self.writer:
                self.writer.add_scalar(f'{model_name}/epoch_loss', avg_loss, epoch)
        
        return model



    def train_skipgram(self, dataloader, epochs=10, model_name="SkipGram"):
        """Обучение SkipGram модели"""
        print(f"Обучение {model_name} модели")
        model = SkipGramModel(self.vocab_size, self.embedding_dim)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=self.learning_rate)
        
        model.train()
        for epoch in range(epochs):
            total_loss = 0
            progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}")
            
            for batch_idx, (target, context) in enumerate(progress_bar):
                optimizer.zero_grad()
                output = model(target)
                loss = criterion(output, context)
                loss.backward()
                optimizer.step()
                
                total_loss += loss.item()
                progress_bar.set_postfix({'loss': f'{loss.item():.4f}'})
                
                if self.writer and batch_idx % 100 == 0:
                    self.writer.add_scalar(f'{model_name}/batch_loss', loss.item(), 
                                         epoch * len(dataloader) + batch_idx)
            
            avg_loss = total_loss / len(dataloader)
            print(f'Epoch {epoch+1}, Average Loss: {avg_loss:.4f}')
            
            if self.writer:
                self.writer.add_scalar(f'{model_name}/epoch_loss', avg_loss, epoch)
        
        return model

In [8]:
DATA_PATH = r'C:\Users\Zver\Desktop\machine_learning\data\Petitions.csv'
SAMPLE_SIZE = 5000 
EMBEDDING_DIM = 200
WINDOW_SIZE = 5
BATCH_SIZE = 128
EPOCHS = 15
MIN_WORD_FREQ = 3

# Создание директории для логов
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
log_dir = f"runs/word2vec_{timestamp}"
os.makedirs(log_dir, exist_ok=True)

print("ЗАГРУЗКА И ПРЕДОБРАБОТКА ДАННЫХ")

print("Загрузка данных")
df = pd.read_csv(DATA_PATH)
texts = df['public_petition_text'].dropna().astype(str).tolist()[:SAMPLE_SIZE]
print(f"Загружено {len(texts)} текстов")

preprocessor = TextPreprocessor()
processed_texts = []

print("Предобработка текстов")
for text in tqdm(texts, desc="Обработка текстов"):
    processed = preprocessor.preprocess(text)
    if len(processed) > 5:  # Пропускаем слишком короткие тексты
        processed_texts.append(processed)

print(f"После фильтрации осталось {len(processed_texts)} текстов")

vocab = Vocabulary(min_freq=MIN_WORD_FREQ)
vocab.build_vocabulary(processed_texts)

ЗАГРУЗКА И ПРЕДОБРАБОТКА ДАННЫХ
Загрузка данных
Загружено 5000 текстов
Предобработка текстов


Обработка текстов: 100%|██████████████████████████████████████████████████████████| 5000/5000 [00:05<00:00, 958.55it/s]


После фильтрации осталось 2328 текстов
Построение словаря


Подсчет частот слов: 100%|█████████████████████████████████████████████████████| 2328/2328 [00:00<00:00, 382240.74it/s]

Размер словаря: 2043 слов
Наиболее частые слова: [('просить', 385), ('убрать', 204), ('дерево', 49), ('кустарник', 11), ('который', 109), ('выйти', 8), ('предел', 10), ('газон', 189), ('пешеходный', 62), ('зона', 25)]


In [9]:
# Создание датасетов и даталоадеров
cbow_dataset = Word2VecDataset(processed_texts, vocab, WINDOW_SIZE, 'cbow')
skipgram_dataset = Word2VecDataset(processed_texts, vocab, WINDOW_SIZE, 'skipgram')

cbow_loader = DataLoader(cbow_dataset, batch_size=BATCH_SIZE, shuffle=True)
skipgram_loader = DataLoader(skipgram_dataset, batch_size=BATCH_SIZE, shuffle=True)

print(f"CBOW samples: {len(cbow_dataset)}")
print(f"SkipGram samples: {len(skipgram_dataset)}")

# Обучение
trainer = Word2VecTrainer(len(vocab), EMBEDDING_DIM, log_dir=log_dir)

cbow_model = trainer.train_cbow(cbow_loader, EPOCHS, "CBOW")
skipgram_model = trainer.train_skipgram(skipgram_loader, EPOCHS, "SkipGram")

Создание датасета для CBOW...


Обработка текстов: 100%|████████████████████████████████████████████████████████| 2328/2328 [00:00<00:00, 13670.38it/s]


Создание датасета для SKIPGRAM...


Обработка текстов: 100%|█████████████████████████████████████████████████████████| 2328/2328 [00:01<00:00, 1248.61it/s]


CBOW samples: 15017
SkipGram samples: 150170
Обучение CBOW модели


...

Epoch 13, Average Loss: 5.0354


Epoch 14/15: 100%|███████████████████████████████████████████████████| 1174/1174 [00:06<00:00, 189.31it/s, loss=5.0475]


Epoch 14, Average Loss: 5.0093


Epoch 15/15: 100%|███████████████████████████████████████████████████| 1174/1174 [00:05<00:00, 204.43it/s, loss=5.1953]

Epoch 15, Average Loss: 4.9867


In [10]:
print("ВИЗУАЛИЗАЦИЯ В TENSORBOARD")

# Создание writer для TensorBoard
writer = SummaryWriter(log_dir)

# Извлечение эмбеддингов
cbow_embeddings = cbow_model.linear.weight.data.cpu()
skipgram_embeddings = skipgram_model.linear.weight.data.cpu()

# Подготовка метаданных
metadata = []
for i in range(len(vocab)):
    word = vocab.idx2word[i]
    freq = vocab.word_freq.get(word, 0)
    metadata.append(f"{word} (freq: {freq})")

print(f"Размер матрицы эмбеддингов: {cbow_embeddings.shape}")
print(f"Количество меток: {len(metadata)}")

# Добавление эмбеддингов в TensorBoard 
print("Добавление эмбеддингов в TensorBoard")

# CBOW эмбеддинги
writer.add_embedding(
    cbow_embeddings,
    metadata=metadata,
    tag="CBOW Embeddings",
    global_step=0
)

# SkipGram эмбеддинги
writer.add_embedding(
    skipgram_embeddings,
    metadata=metadata,
    tag="SkipGram Embeddings", 
    global_step=0
)

# PCA
pca = PCA(n_components=50)
cbow_pca = pca.fit_transform(cbow_embeddings.numpy())
skipgram_pca = pca.fit_transform(skipgram_embeddings.numpy())

writer.add_embedding(
    torch.tensor(cbow_pca),
    metadata=metadata,
    tag="CBOW PCA-50",
    global_step=0
)

writer.add_embedding(
    torch.tensor(skipgram_pca),
    metadata=metadata,
    tag="SkipGram PCA-50",
    global_step=0
)

writer.close()

ВИЗУАЛИЗАЦИЯ В TENSORBOARD
Размер матрицы эмбеддингов: torch.Size([2043, 200])
Количество меток: 2043
Добавление эмбеддингов в TensorBoard


In [11]:
# Сохранение моделей
torch.save({
    'cbow_model': cbow_model.state_dict(),
    'skipgram_model': skipgram_model.state_dict(),
    'vocab': vocab,
    'embeddings_dim': EMBEDDING_DIM
}, f'{log_dir}/word2vec_models.pth')

In [12]:
%load_ext tensorboard
%tensorboard --logdir=runs